In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime
import pymysql

In [25]:

file_path = r"C:\Users\sugawanesh\Downloads\traffic_stops - traffic_stops_with_vehicle_number (1).csv"
df = pd.read_csv(file_path)

C:\Users\sugawanesh\AppData\Local\Temp\ipykernel_21104\640362261.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [26]:
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [27]:
df['stop_date'] = pd.to_datetime(df['stop_date'], format="%Y-%m-%d")
df['stop_time'] = pd.to_datetime(df['stop_time'], format="%H:%M:%S")

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.drop('driver_age_raw', axis=1, inplace=True)

df.fillna({'search_type':'None'}, inplace=True)
df.isnull().sum()


stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64

In [ ]:
import pymysql


try:
    conn = pymysql.connect(
        host='localhost',
        user='root',
        password='sugu', 
        database='policelogs'      
    )
    
   
    cursor = conn.cursor()
    print("Connection and cursor ready!")


    cursor.execute("""
    CREATE TABLE IF NOT EXISTS public_safety (
        id int auto_increment primary key,
        stop_date DATE,
        stop_time TIME,
        country_name varchar(50),
        driver_gender varchar(2),
        driver_age int,
        driver_race varchar(50),
        violation varchar(50),
        search_conducted boolean,
        search_type varchar(50),
        stop_outcome varchar(50),
        is_arrested boolean,
        stop_duration varchar(50),
        drugs_related_stop boolean,
        vehicle_number varchar(50),
        timestamp datetime
    )
    """)
    print("Table created successfully!")

except Exception as e:
    print(f"An error occurred: {e}")

Connection and cursor ready!
Table created successfully!


In [ ]:

from datetime import datetime
df["timestamp"] = datetime.now()
insert_values = df[
    [
        "stop_date",
        "stop_time",
        "country_name",
        "driver_gender",
        "driver_age",
        "driver_race",
        "violation",
        "search_conducted",
        "search_type",
        "stop_outcome",
        "is_arrested",
        "stop_duration",
        "drugs_related_stop",
        "vehicle_number",
        "timestamp"
    ]
].values.tolist()
conn.commit()

query = "insert into public_safety (stop_date, stop_time, country_name, driver_gender, driver_age, driver_race, violation, search_conducted, search_type, stop_outcome, is_arrested, stop_duration, drugs_related_stop,vehicle_number,timestamp) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query, insert_values)
print("Insert values successfully")

Insert values successfully
